In [1]:
#use below command to wake up the connection between Django and Jupyter:
# my_project_env/Scripts/activate      
# python manage.py shell_plus -- notebook 

In [1]:
import sys
sys.path.append('C:/Users/das/Desktop/Workspace/isaac-e-clipse/isaac-dvadmin-backend/backend')
# sys.path.append('C:/Users/das/Desktop/Workspace/isaac-e-clipse/isaac-dvadmin-backend/backend/project/views/calculation/requirements.txt')
! pip install -r requirements.txt

Looking in indexes: https://artifactory-esc.corp.knorr-bremse.com:8482/artifactory/api/pypi/py-python.org/simple/


In [4]:
json_data = '''
{
  "leftForm": {
    "caliperType": "compact",
    "caliperTypeDesignation": "WZKT (II108615_6027YM) ",
    "discType": 2,
    "discProduct": 10,
    "discInnerDiameter": "350.1111",
    "discOuterDiameter": 640,
    "wheelInnerDiameter": 720,
    "discWidth": 135,
    "padShape": "FLEX400KRS",
    "padShapeStepFile": "FLEX400KRS_SINTER_II104295_(67.9).stp",
    "padType": "1",
    "padProduct": 6,
    "caliperDetailType": 5,
    "caliperProduct": 13,
    "padThickness": 24,
    "clearance": 1.5,
    "eDimension": 245,
    "ioy": 2,
    "ioz": 5,
    "roc": 5,
    "discWear": 5,
    "padWear": 19,
    "xlm": 5,
    "ylm": 5,
    "atd": 2,
    "btd": -45,
    "ctd": 2,
    "dac": 2,
    "eat": 2,
    "sideReference": "eccentricSide",
    "wheelSettingMode": "convexCurve",
    "discWearStart": 712,
    "discWearEnd": 715,
    "wearOfLeverBolt": 2,
    "wearOfPadHolderBolt": 3.27,
    "hangerUpperBoltToCaliperCenter": 319.5,
    "hangerLowerBoltToPadCenter": 160
  },
  "rightForm": {
    "oneStep": {
      "wheelSettingMode": "eccentricShaftSide",
      "padHolderBoltWear":0,
      "leverBoltWear":0,
      "verticalMove": 0,
      "lateralMove": 0,
      "longMove": 0,
      "padWear": 19,
      "discWear": 5,
      "fPointOffset": 0,
      "brakingApplied": "yes"
    },
    "odyssey": {
      "wheelSettingMode": "eccentricShaftSide",
      "tareTCrush": 0,
      "lateralMove": 0,
      "liftStop": 0,
      "long": 0,
      "bumpStop": 0,
      "longMove": 0,
      "padWear": 19,
      "discWear": 5,
      "steps": 0
    }
  },
  "activeName": "One Step"
}
'''

In [5]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"]="true"

In [6]:
from asgiref.sync import sync_to_async
import nest_asyncio
nest_asyncio.apply()

import sys
sys.path.append('C:/Users/das/Desktop/Workspace/isaac-e-clipse/isaac-dvadmin-backend/backend')
import os  
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'application.settings')  

import django  
try:  
    django.setup()  
except Exception as e:  
    print(f"Error initializing Django: {e}")


In [12]:
import json
from dataclasses import dataclass

@dataclass
class Component:
    pass

@dataclass
class Actuator(Component):
    caliperType: str
    caliperDetailType: str
    caliperItem: str

@dataclass
class Disc(Component):
    discInnerDiameter: float
    discOuterDiameter: float
    discWidth: float

@dataclass
class Wheel(Component):
    wheelInnerDiameter: float

@dataclass
class Pad(Component):
    padId: int  # padId
    padThickness: float
    clearance: float

@dataclass
class BrakeCaliperPositioning(Component):
    eDimension: float
    installationOffsetY: float  # installationOffsetY
    installationOffsetZ: float  # installationOffsetZ
    rotationOfCaliper: float  # rotationOfCaliper
    discWear: float
    padWear: float

@dataclass
class KinematicPrimarySuspension(Component):
    X_LongitudinalMovement: float  # x_LongitudinalMovement
    Y_LateralMovement: float  # y_LateralMovement
    A_TareDatumToLiftStop: float  # a_TareDatumToLiftStop
    B_TareDatumToBumpStop: float  # b_TareDatumToBumpStop
    C_TareDatumToCrushDatum: float  # C_TareDatumToCrushDatum
    D_AmplitudeCrush: float  # D_AmplitudeCrush
    E_AmplitudeTare: float  # E_AmplitudeTare
    discWearStart: float
    discWearEnd: float
    wearOfLeverBolt: float
    wearOfPadHolderBolt: float

@dataclass
class FunctionalInputs(Component):
    inputs: dict

@dataclass
class ExtraSettings(Component):
    sideReference: str
    wheelSettingMode: str


@dataclass
class BogieBrakeSystem:
    left_form: dict
    right_form: dict
    active_name: str

    def __post_init__(self):
        try:
            # 提取并初始化组件
            self.actuator = Actuator(
                caliperType=self.left_form['caliperType'],
                caliperDetailType=self.left_form['caliperDetailType'],
                caliperItem=self.left_form['caliperProduct']
            )
            self.disc = Disc(
                discInnerDiameter=float(self.left_form['discInnerDiameter']),
                discOuterDiameter=self.left_form['discOuterDiameter'],
                discWidth=self.left_form['discWidth']
            )
            self.wheel = Wheel(
                wheelInnerDiameter=self.left_form['wheelInnerDiameter']
            )
            self.pad = Pad(
                padId=int(self.left_form['padProduct']),  # 将 padProduct 转换为整数
                padThickness=self.left_form['padThickness'],
                clearance=self.left_form['clearance']
            )
            self.brakeCaliperPositioning = BrakeCaliperPositioning(
                eDimension=self.left_form['eDimension'],
                installationOffsetY=self.left_form['ioy'],
                installationOffsetZ=self.left_form['ioz'],
                rotationOfCaliper=self.left_form['roc'],
                discWear=self.left_form['discWear'],
                padWear=self.left_form['padWear']
            )
            self.kinematicPrimarySuspension = KinematicPrimarySuspension(
                X_LongitudinalMovement=self.left_form['xlm'],
                Y_LateralMovement=self.left_form['ylm'],
                A_TareDatumToLiftStop=self.left_form['atd'],
                B_TareDatumToBumpStop=self.left_form['btd'],
                C_TareDatumToCrushDatum=self.left_form['ctd'],
                D_AmplitudeCrush=self.left_form['dac'],
                E_AmplitudeTare=self.left_form['eat'],
                discWearStart=self.left_form['discWearStart'],
                discWearEnd=self.left_form['discWearEnd'],
                wearOfLeverBolt=self.left_form['wearOfLeverBolt'],
                wearOfPadHolderBolt=self.left_form['wearOfPadHolderBolt']
            )
            self.extraSettings = ExtraSettings(
                sideReference=self.left_form['sideReference'],
                wheelSettingMode=self.left_form['wheelSettingMode']
            )
            # self.right_form = ExtraSettings(
            #     sideReference=self.left_form['sideReference'],
            #     wheelSettingMode=self.left_form['wheelSettingMode']
            # )
            # # 根据 active_name 选择正确的配置
            # if self.active_name == "One Step":
            #     self.functionalInputs = self.right_form.get("oneStep", {})
            # elif self.active_name == "Odyssey":
            #     self.functionalInputs = self.right_form.get("odyssey", {})
            # else:
            self.functionalInputs = FunctionalInputs(
                inputs=self.right_form
            )
        except KeyError as e:
            print(f"Missing key in form data: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

In [13]:
data = json.loads(json_data)
bogie_brake_system = BogieBrakeSystem(
    left_form=data["leftForm"],
    right_form=data["rightForm"],
    active_name=data["activeName"]
)
type(bogie_brake_system.disc.discInnerDiameter)
# actuator_data=Brake.objects.get(id=2)
# bogie_brake_system.functionalInputs.inputs

float

In [7]:
from abc import ABC, abstractmethod  
from collections import namedtuple
import math
from project.models import *
from functools import cached_property
import time
import numpy as np
# 加载 JSON 数据


# TransferResult = namedtuple('TransferResult', [
#     'G', 'R', 'E_other', 'D_w_l_min', 'D_w_l_max', 'Delta_Z', 'Delta_Zmax', 'Delta_Zmin',
#     'D_w_l_min_L0', 'D_w_l_max_L0', 'D_w_l_w_min', 'D_w_l_w_max', 'Delta_Z_D_min',
#     'Delta_Z_D_max', 'comment'
# ])
# TransferInstantResult = namedtuple('TransferInstantResult', ['Delta_Z','D_w_l_Instant','Delta_Z_Instant'
# ])
@dataclass  
class TransferResult:  
    G: float  
    R: float  
    E_other: float  
    D_w_l_min: float  
    D_w_l_max: float  
    Delta_Z: float  
    Delta_Zmax: float  
    Delta_Zmin: float  
    D_w_l_min_L0: float  
    D_w_l_max_L0: float  
    D_w_l_w_min: float  
    D_w_l_w_max: float  
    Delta_Z_D_min: float  
    Delta_Z_D_max: float  
    comment: str  # 假设 comment 是可选的  
  
@dataclass  
class TransferInstantResult:  
    Delta_Z: float  
    D_w_l_Instant: float  
    Delta_Z_Instant: float
# Base class for the strategy pattern


class EclipseCalculationUtils:
    @staticmethod
    def Rm_cal(P_points, P_XY, E_dim, E_delta_Rm, V_Rm):
        # a_com_rm = 0
        # r_sx_rm = 0
        # I_y_rm = 0
        # n_area=len(P_points)

        # for i in range(0, n_area):
        #     n_points = P_points[i]
        #     for j in range(0, n_points):
        #         P_XY[i][j][0] = P_XY[i][j][0] + E_delta_Rm
        #         P_XY[i][j][1] = P_XY[i][j][1] + V_Rm

        # for i in range(0, n_area):
        #     n_points = P_points[i]
        #     for j in range(0, n_points - 1):

        #         d_x_i_rm = P_XY[i][j + 1][0] - P_XY[i][j][0]
        #         d_y_i_rm = (P_XY[i][j + 1][1] + P_XY[i][j][1]) / 2
        #         r_sx_i_rm = (P_XY[i][j + 1][0] + P_XY[i][j][0]) / 2
        #         r_sy_i_rm = (P_XY[i][j + 1][1] + P_XY[i][j][1]) / 4
        #         a_i_rm = d_x_i_rm * d_y_i_rm
        #         a_com_rm = a_com_rm + a_i_rm
        #         r_sx_rm = r_sx_rm + r_sx_i_rm * a_i_rm
        #         I_y_i_rm = d_y_i_rm * d_x_i_rm ** 3 / 12
        #         I_y_rm = I_y_i_rm + I_y_rm + r_sx_i_rm ** 2 * a_i_rm

        # r_sx_rm = r_sx_rm / a_com_rm
        # I_y_rm = I_y_rm - r_sx_rm ** 2 * a_com_rm


        # Rm = 0
        # n_p_rm = 30
        # r_fx_rmm = E_dim+ E_delta_Rm
        # for i in range(0, n_area):
        #     n_points = P_points[i]
        #     for j in range(0, n_points - 1):
        #         d_x_i_rm = (P_XY[i][j + 1][0] - P_XY[i][j][0]) / n_p_rm
        #         d_y_i_rm = (P_XY[i][j + 1][1] + P_XY[i][j][1]) / (2 * n_p_rm)
        #         a_i_rm = d_x_i_rm * d_y_i_rm

        #         for l_rm in range(0, n_p_rm):
        #             r_sx_i_rm = P_XY[i][j][0] + d_x_i_rm * (l_rm + 0.5)
        #             for k_rm in range(0, n_p_rm):
        #                 r_sy_i_rm = d_y_i_rm * (k_rm + 0.5)
        #                 lamda_rm = (1 / a_com_rm + (r_fx_rmm - r_sx_rm) * (r_sx_i_rm - r_sx_rm) / I_y_rm) * (r_sx_i_rm ** 2 + r_sy_i_rm ** 2) ** 0.5
        #                 Rm = Rm + lamda_rm * a_i_rm
        # Rm = round(Rm, 1)
        P_XY = np.array(P_XY)
        P_XY[:, :, 0] += E_delta_Rm
        P_XY[:, :, 1] += V_Rm
        a_com_rm = 0
        r_sx_rm = 0
        I_y_rm = 0
        
        for i in range(len(P_points)):
            n_points = P_points[i]
            d_x_i_rm = P_XY[i, 1:n_points, 0] - P_XY[i, :n_points-1, 0]
            d_y_i_rm = (P_XY[i, 1:n_points, 1] + P_XY[i, :n_points-1, 1]) / 2
            r_sx_i_rm = (P_XY[i, 1:n_points, 0] + P_XY[i, :n_points-1, 0]) / 2
            r_sy_i_rm = (P_XY[i, 1:n_points, 1] + P_XY[i, :n_points-1, 1]) / 4
            a_i_rm = d_x_i_rm * d_y_i_rm
            a_com_rm += np.sum(a_i_rm)
            r_sx_rm += np.sum(r_sx_i_rm * a_i_rm)
            I_y_i_rm = d_y_i_rm * d_x_i_rm ** 3 / 12
            I_y_rm += np.sum(I_y_i_rm + r_sx_i_rm ** 2 * a_i_rm)
        
        r_sx_rm /= a_com_rm
        I_y_rm -= r_sx_rm ** 2 * a_com_rm
        
        Rm = 0
        n_p_rm = 30
        r_fx_rmm = E_dim + E_delta_Rm
        
        for i in range(len(P_points)):
            n_points = P_points[i]
            d_x_i_rm = (P_XY[i, 1:n_points, 0] - P_XY[i, :n_points-1, 0]) / n_p_rm
            d_y_i_rm = (P_XY[i, 1:n_points, 1] + P_XY[i, :n_points-1, 1]) / (2 * n_p_rm)
            a_i_rm = d_x_i_rm * d_y_i_rm
            
            for l_rm in range(n_p_rm):
                r_sx_i_rm = P_XY[i, :n_points-1, 0] + d_x_i_rm * (l_rm + 0.5)
                for k_rm in range(n_p_rm):
                    r_sy_i_rm = d_y_i_rm * (k_rm + 0.5)
                    lamda_rm = (1 / a_com_rm + (r_fx_rmm - r_sx_rm) * (r_sx_i_rm - r_sx_rm) / I_y_rm) * np.sqrt(r_sx_i_rm ** 2 + r_sy_i_rm ** 2)
                    Rm += np.sum(lamda_rm * a_i_rm)
  
        Rm = round(Rm, 1)

        return Rm

    @staticmethod
    def innermost(array_group):
        # point = min(list(map(lambda x: min(x, key=lambda x: x[0] ** 2 + x[1] ** 2), array_group)),
        #                     key=lambda x: x[0] ** 2 + x[1] ** 2)
        # return point
        concatenated = np.concatenate(array_group)  

        distances_squared = concatenated[:, 0]**2 + concatenated[:, 1]**2  
 
        min_index = np.argmin(distances_squared)  

        return concatenated[min_index].tolist()  
    
    @staticmethod
    def outermost(array_group):
        # point = max(list(map(lambda x: max(x, key=lambda x: x[0] ** 2 + x[1] ** 2), array_group)),
        #                     key=lambda x: x[0] ** 2 + x[1] ** 2)
        # return point
  
        concatenated = np.concatenate(array_group)  
  
        distances_squared = concatenated[:, 0]**2 + concatenated[:, 1]**2  
 
        max_index = np.argmax(distances_squared)  

        return concatenated[max_index].tolist()   
     
    @staticmethod
    def circle_array(diameter):

        radius = diameter / 2
        # 创建一个从0到1的100个点的数组
        theta = np.linspace(0, 2 * np.pi, 100)
        # 计算圆上的点的坐标
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        # 将计算结果转换为列表
        array_circle = list(zip(x, y))

        return array_circle

class BXActCalculatorBase(ABC):
    def __init__(self, E, angle, Pad_thickness, clearance, y_offset, Disc_Width, y_move,
                 disc_wear, pad_wear):
        self.E = E  # The installation reference dimension for Caliper/TBU
        self.angle = angle
        self.Pad_thickness = Pad_thickness
        self.clearance = clearance
        self.y_offset = y_offset
        self.Disc_Width = Disc_Width
        self.y_move = y_move
        self.disc_wear = disc_wear
        self.pad_wear = pad_wear

    @abstractmethod
    def transfer_list(self):
        pass
    @abstractmethod
    def transfer_list_instant(self):
        pass

class BXCalculationModeBase(ABC):

    @abstractmethod
    def calculation_results():
        pass

class ClassicActCalculator(BXActCalculatorBase):
    def __init__(self, E, angle, Pad_thickness, clearance, y_offset, Disc_Width, y_move,
                 disc_wear, pad_wear, Lever_length, LLF, Offset_Lever,
                     PHH, WLB, WPHB, DCY, PAC, G_dimension, HTL, BFH,
                     DHB, HUTC, HLPC):
        super().__init__(E, angle, Pad_thickness, clearance, y_offset, Disc_Width, y_move,
                     disc_wear, pad_wear)
        self.Lever_length = Lever_length
        self.LLF = LLF
        self.Offset_Lever = Offset_Lever
        self.PHH = PHH
        self.WLB = WLB
        self.WPHB = WPHB
        self.DCY = DCY
        self.PAC = PAC
        self.G_dimension = G_dimension
        self.HTL = HTL
        self.BFH = BFH
        self.DHB = DHB
        self.HUTC = HUTC
        self.HLPC = HLPC
    @property
    def transfer_list(self):
        G30 = self.Disc_Width + 2 * (self.Pad_thickness + self.PHH + self.clearance)
        G = self.DCY + self.PAC + (self.Lever_length - self.LLF) / self.Lever_length * (
                    G30 - (self.DCY + self.PAC)) + 2 * self.Offset_Lever
        RR = ((self.LLF) ** 2 + (self.Offset_Lever) ** 2) ** 0.5
        L_0 = self.y_offset
        L_L = self.y_move
        G_W = self.WLB
        D_w = self.pad_wear + self.disc_wear + self.WPHB
        Lb = G30
        CR = self.clearance

        yyt = (G - Lb) / 2 + L_0
        xxt = (RR ** 2 - yyt ** 2) ** 0.5

        y_0 = yyt

        yyt_other = (G - Lb) / 2 - L_0
        xxt_other = (RR ** 2 - yyt_other ** 2) ** 0.5
        E_other = round(self.E + xxt - xxt_other, 2)

        y_low = yyt - abs(L_L) + CR
        y_up = (G + G_W - Lb) / 2 + L_0 + abs(L_L) + D_w + CR
        y_low_t = y_low

        if y_low * y_up > 0:
            y_low = y_low
        else:
            y_low = 0
        y_min = min(abs(y_low), abs(y_up))
        y_max = max(abs(y_low_t), abs(y_up))

        if y_min == abs(y_low):
            y_min_with = y_low
        else:
            y_min_with = y_up - self.WPHB - G_W / 2

        if y_max == abs(y_low_t):
            y_max_with = y_low_t
        else:
            y_max_with = y_up - self.WPHB - G_W / 2

        D_w_l_min = xxt - (RR ** 2 - y_min ** 2) ** 0.5
        D_w_l_max = xxt - (RR ** 2 - y_max ** 2) ** 0.5

        if y_max == abs(y_low_t):
            y_Emax = y_low_t
        else:
            y_Emax = y_up

        if y_min == abs(y_low):
            y_Emin = y_low
        else:
            y_Emin = y_up

        y_up_N1 = (G + G_W - Lb) / 2 + L_0 + abs(L_L) + D_w + CR - self.disc_wear
        y_max_N1 = max(abs(y_low_t), abs(y_up_N1))

        D_w_l_w_min = xxt - (RR ** 2 - y_max_N1 ** 2) ** 0.5

        if y_max_N1 == abs(y_low_t):
            y_max_with_N1 = y_low_t
            comment = f" Disc wear:0 \n Pad wear:0 \n Latateral move.:{-abs(L_L)}"

        else:
            y_max_with_N1 = y_up_N1 - self.WPHB - G_W / 2
            comment = f" Disc wear:0 \n Maximum Pad wear \n Latateral move.:{abs(L_L)}"

        yyt_disc_wear = (G - Lb + 2 * self.disc_wear) / 2 + L_0

        y_low_t_N2 = yyt_disc_wear - abs(L_L) + CR
        y_up_N2 = (G + G_W - Lb) / 2 + L_0 + abs(L_L) + D_w + CR

        y_max_N2 = max(abs(y_low_t_N2), abs(y_up_N2))

        D_w_l_w_max = xxt - (RR ** 2 - y_max_N2 ** 2) ** 0.5

        if y_max_N2 == abs(y_low_t_N2):
            y_max_with_N2 = y_low_t_N2
            comment = f" Maximum Disc wear \n Pad wear:0 \n Latateral move.:{-abs(L_L)}"

        else:
            y_max_with_N2 = y_up_N2 - self.WPHB - G_W / 2
            comment = f" Maximum Disc wear \n Maximum Pad wear \n Latateral move.:{abs(L_L)}"

        y_low_L0 = yyt + CR
        y_up_L0 = (G + G_W - Lb) / 2 + L_0 + D_w + CR
        y_low_L0_t = y_low_L0

        if y_low_L0 * y_up_L0 > 0:
            y_low_L0 = y_low_L0
        else:
            y_low_L0 = 0

        y_min = min(abs(y_low_L0), abs(y_up_L0))
        y_max = max(abs(y_low_L0_t), abs(y_up_L0))

        D_w_l_min_L0 = xxt - (RR ** 2 - y_min ** 2) ** 0.5
        D_w_l_max_L0 = xxt - (RR ** 2 - y_max ** 2) ** 0.5

        if y_max == abs(y_low_L0_t):
            y_Emax_D = y_low_L0_t
        else:
            y_Emax_D = y_up_L0

        if y_min == abs(y_low_L0):
            y_Emin_D = y_low_L0
        else:
            y_Emin_D = y_up_L0

        HUC = self.HUTC
        HLP = self.HLPC
        HLI = self.HTL
        HZR = HUC
        HBM = 2 * self.BFH + 2 * self.Pad_thickness + self.Disc_Width + 2 * CR
        HX = self.DHB

        HZ0 = (HLI ** 2 - ((HBM - HX) / 2) ** 2) ** 0.5 + HLP

        Delta_Z = HZR - HZ0

        HZ0 = (HLI ** 2 - ((HBM - HX) / 2) ** 2) ** 0.5

        HZ = (HLI ** 2 - ((HBM - HX) / 2 + yyt - y_max_with) ** 2) ** 0.5

        Delta_Zmax = HZ0 - HZ

        HZ = (HLI ** 2 - ((HBM - HX) / 2 + yyt - y_min_with) ** 2) ** 0.5

        Delta_Zmin = HZ0 - HZ

        HZ_N1 = (HLI ** 2 - ((HBM - HX) / 2 + yyt - y_max_with_N1) ** 2) ** 0.5
        Delta_Z_D_min = HZ0 - HZ_N1
        HZ_N2 = (HLI ** 2 - ((HBM - HX) / 2 + yyt - y_max_with_N2) ** 2) ** 0.5
        Delta_Z_D_max = HZ0 - HZ_N2

        return TransferResult(
            G=G,
            R=round(RR, 2),
            E_other=E_other,
            D_w_l_min=D_w_l_min,
            D_w_l_max=D_w_l_max,
            Delta_Z=Delta_Z,
            Delta_Zmax=Delta_Zmax,
            Delta_Zmin=Delta_Zmin,
            D_w_l_min_L0=D_w_l_min_L0,
            D_w_l_max_L0=D_w_l_max_L0,
            D_w_l_w_min=D_w_l_w_min,
            D_w_l_w_max=D_w_l_w_max,
            Delta_Z_D_min=Delta_Z_D_min,
            Delta_Z_D_max=Delta_Z_D_max,
            comment=comment
        )

    def transfer_list_instant(self, y_move_instant, disc_wear_instant, pad_wear_instant, WLB_instant, WPHB_instant):
        G30 = self.Disc_Width + 2 * (self.Pad_thickness + self.PHH + self.clearance)
        G = self.DCY + self.PAC + (self.Lever_length - self.LLF) / self.Lever_length * (
                G30 - (self.DCY + self.PAC)) + 2 * self.Offset_Lever
        RR = ((self.LLF) ** 2 + (self.Offset_Lever) ** 2) ** 0.5
        L_0 = self.y_offset
        L_L_instant = y_move_instant
        G_W_instant = WLB_instant
        D_w_instant = pad_wear_instant + disc_wear_instant + WPHB_instant
        Lb = G30
        CR = self.clearance

        yyt = (G - Lb) / 2 + L_0
        xxt = (RR ** 2 - yyt ** 2) ** 0.5

        y_instant = (G + G_W_instant - Lb) / 2 + L_0 - L_L_instant + D_w_instant + CR
        D_w_l_Instant = xxt - (RR ** 2 - y_instant ** 2) ** 0.5

        HUC = self.HUTC
        HLP = self.HLPC
        HLI = self.HTL
        HZR = HUC
        HBM = 2 * self.BFH + 2 * self.Pad_thickness + self.Disc_Width + 2 * CR
        HX = self.DHB

        HZ0 = (HLI ** 2 - ((HBM - HX) / 2) ** 2) ** 0.5 + HLP

        Delta_Z = HZR - HZ0

        HZ0 = (HLI ** 2 - ((HBM - HX) / 2) ** 2) ** 0.5

        HZ = (HLI ** 2 - ((HBM - HX) / 2 + L_L_instant - D_w_instant - CR + WPHB_instant) ** 2) ** 0.5

        Delta_Z_Instant = HZ0 - HZ

        return TransferInstantResult(
            Delta_Z=Delta_Z,
            D_w_l_Instant=D_w_l_Instant,
            Delta_Z_Instant=Delta_Z_Instant
        )

class CompactActCalculator(BXActCalculatorBase):
    def __init__(self, E, angle, Pad_thickness, clearance, y_offset, Disc_Width, y_move,
                 disc_wear, pad_wear, LL, LR, LR1,
                     LR2, D1, D2, D3, H1, H2, H3,Side):
        super().__init__(E, angle, Pad_thickness, clearance, y_offset, Disc_Width, y_move,
                     disc_wear, pad_wear)
        self.LL = LL
        self.LR = LR
        self.LR1 = LR1
        self.LR2 = LR2
        self.D1 = D1
        self.D2 = D2
        self.D3 = D3
        self.H1 = H1
        self.H2 = H2
        self.H3 = H3
        self.Side = Side

    @property
    def transfer_list(self):
        E = self.E
        LL = self.LL
        LR = self.LR
        LR1 = self.LR1
        LR2 = self.LR2
        D1 = self.D1
        D2 = self.D2
        D3 = self.D3
        H1 = self.H1
        H2 = self.H2
        H3 = self.H3
        C = self.clearance
        disc_wear = self.disc_wear
        L_0 = self.y_offset
        L_L = -1 * self.y_move
        D_w = disc_wear + self.pad_wear
        Side = self.Side

        if Side == 1:
            yyt = LR + ((LR2) ** 2 - (LR1) ** 2) ** 0.5 - C - H3 / 2 - H1 - H2 + L_0
            yyt_other = LL - C - H3 / 2 - H1 - H2 - L_0
        else:
            yyt = LL - C - H3 / 2 - H1 - H2 + L_0
            yyt_other = LR + ((LR2) ** 2 - (LR1) ** 2) ** 0.5 - C - H3 / 2 - H1 - H2 - L_0
        y_0 = yyt
        xxt = ((D1) ** 2 - (yyt) ** 2) ** 0.5
        xxt_other = ((D1) ** 2 - (yyt_other) ** 2) ** 0.5
        E_other = round(E + xxt - xxt_other, 2)
        RR = D1
        CR = C
        # '===============================
        # 'D_w_l_min, D_w_l_max
        # '===============================
        y_low = yyt - abs(L_L) + CR
        y_up = yyt + abs(L_L) + D_w + CR
        y_low_t = y_low
        if y_low * y_up > 0:
            y_low = y_low
        else:
            y_low = 0

        y_min = min(abs(y_low), abs(y_up))
        y_max = max(abs(y_low_t), abs(y_up))

        if y_max == abs(y_low_t):
            y_Emax = y_low_t
        else:
            y_Emax = y_up

        if y_min == abs(y_low):
            y_Emin = y_low
        else:
            y_Emin = y_up

        D_w_l_min = xxt - (RR ** 2 - y_min ** 2) ** 0.5
        D_w_l_max = xxt - (RR ** 2 - y_max ** 2) ** 0.5

        # 'NEW
        # '===============================
        # 'NEW                 D_w_l_w_min, D_w_l_w_max
        # '===============================
        # 'NEW
        y_up_N1 = y_up - disc_wear
        y_max_N1 = max(abs(y_low_t), abs(y_up_N1))
        D_w_l_w_min = xxt - (RR ** 2 - y_max_N1 ** 2) ** 0.5

        if y_max_N1 == abs(y_low_t):
            # y_max_with_N1 = y_low_t
            comment = f" Disc wear:0 \n Pad wear:0 \n Latateral move.:{-abs(L_L)}"

        else:
            # y_max_with_N1 = y_up_N1 - self.WPHB - G_W / 2
            comment = f" Disc wear:0 \n Maximum Pad wear \n Latateral move.:{abs(L_L)}"

        yyt_disc_wear = yyt + disc_wear
        y_low_t_N2 = yyt_disc_wear - abs(L_L) + CR
        y_up_N2 = y_up
        y_max_N2 = max(abs(y_low_t_N2), abs(y_up_N2))
        D_w_l_w_max = xxt - (RR ** 2 - y_max_N2 ** 2) ** 0.5

        if y_max_N2 == abs(y_low_t_N2):
            # y_max_with_N2 = y_low_t_N2
            comment = f" Maximum Disc wear \n Pad wear:0 \n Latateral move.:{-abs(L_L)}"
        else:
            comment = f" Maximum Disc wear \n Maximum Pad wear \n Latateral move.:{abs(L_L)}"

        y_low_L0 = yyt + CR
        y_up_L0 = yyt + D_w + CR
        y_low_L0_t = y_low_L0

        if y_low_L0 * y_up_L0 > 0:
            y_low_L0 = y_low_L0
        else:
            y_low_L0 = 0

        y_min = min(abs(y_low_L0), abs(y_up_L0))
        y_max = max(abs(y_low_L0_t), abs(y_up_L0))

        D_w_l_min_L0 = xxt - (RR ** 2 - y_min ** 2) ** 0.5
        D_w_l_max_L0 = xxt - (RR ** 2 - y_max ** 2) ** 0.5

        if y_max == abs(y_low_L0_t):
            y_Emax_D = y_low_L0_t
        else:
            y_Emax_D = y_up_L0

        if y_min == abs(y_low_L0):
            y_Emin_D = y_low_L0
        else:
            y_Emin_D = y_up_L0

        Delta_Z = 0
        Delta_Zmax = 0
        Delta_Zmin = 0
        Delta_Z_D_min = 0
        Delta_Z_D_max = 0

        return TransferResult(
            G=0,
            R=round(RR, 2),
            E_other=E_other,
            D_w_l_min=D_w_l_min,
            D_w_l_max=D_w_l_max,
            Delta_Z=Delta_Z,
            Delta_Zmax=Delta_Zmax,
            Delta_Zmin=Delta_Zmin,
            D_w_l_min_L0=D_w_l_min_L0,
            D_w_l_max_L0=D_w_l_max_L0,
            D_w_l_w_min=D_w_l_w_min,
            D_w_l_w_max=D_w_l_w_max,
            Delta_Z_D_min=Delta_Z_D_min,
            Delta_Z_D_max=Delta_Z_D_max,
            comment=comment
        )

    def transfer_list_instant(self, y_move_instant, disc_wear_instant, pad_wear_instant, WLB_instant=0, WPHB_instant=0):
        LR = self.LR
        LR1 = self.LR1
        LR2 = self.LR2
        D1 = self.D1
        D2 = self.D2
        D3 = self.D3
        H1 = self.H1
        H2 = self.H2
        H3 = self.H3
        C = self.clearance
        L_0 = self.y_offset

        Side = self.Side
        L_L_instant = y_move_instant
        D_w_instant = pad_wear_instant + disc_wear_instant

        if Side == 1:
            yyt = LR + ((LR2) ** 2 - (LR1) ** 2) ** 0.5 - C - H3 / 2 - H1 - H2 + L_0
        else:
            yyt = L_L_instant - C - H3 / 2 - H1 - H2 + L_0
        y_0 = yyt
        xxt = ((D1) ** 2 - (yyt) ** 2) ** 0.5
        RR = D1
        CR = C

        y_instant = yyt - L_L_instant + CR + D_w_instant
        D_w_l_Instant = xxt - (RR ** 2 - y_instant ** 2) ** 0.5
        Delta_Z = 0
        Delta_Z_Instant = 0

        return TransferInstantResult(
            Delta_Z=Delta_Z,
            D_w_l_Instant=D_w_l_Instant,
            Delta_Z_Instant=Delta_Z_Instant
        )

class BXCalculationSystem:

    def __init__(self,json_data):
        data = json.loads(json_data)
        self.bogie_brake_system = BogieBrakeSystem(
            left_form=data["leftForm"],
            right_form=data["rightForm"],
            active_name=data["activeName"]
        )
        self.kwargs={}
        self.actuator_object=self.actuator_object
        self.pad_object=self.pad_object

    @cached_property
    def actuator_object(self):
        return Brake.objects.get(id=self.bogie_brake_system.actuator.caliperItem) 
       
    @cached_property
    def pad_object(self):
        return BrakeLiningShoe.objects.get(id=self.bogie_brake_system.pad.padId)    

    @cached_property
    def actuator_calculator(self):     
        # get Actuator object by ORM
        actuator_data=self.actuator_object
        actuator_type = self.bogie_brake_system.actuator.caliperType
        self.kwargs = {
            'E': self.bogie_brake_system.brakeCaliperPositioning.eDimension,
            'y_offset': self.bogie_brake_system.brakeCaliperPositioning.installationOffsetY,
            'angle': self.bogie_brake_system.brakeCaliperPositioning.rotationOfCaliper,
            'Disc_Width': self.bogie_brake_system.disc.discWidth,
            'y_move': self.bogie_brake_system.brakeCaliperPositioning.rotationOfCaliper,
            'disc_wear': self.bogie_brake_system.brakeCaliperPositioning.discWear,
            'pad_wear': self.bogie_brake_system.brakeCaliperPositioning.padWear,
            'clearance': self.bogie_brake_system.pad.clearance,
            'Pad_thickness': self.bogie_brake_system.pad.padThickness
            # Additional parameters for ClassicAct or CompactAct
        }
        
        if actuator_type == 'classic':
            additional_kwargs = {
                'Lever_length': actuator_data.lever_length,
                'LLF': actuator_data.lever_length_the_front,
                'Offset_Lever': actuator_data.offset_pull_rod_rotation_point_in_lever,
                'PHH': actuator_data.pad_holder_height,
                'WLB': actuator_data.wear_of_lever_bolt,
                'WPHB': actuator_data.wear_of_pad_holder_bolt,
                'DCY': actuator_data.distance_cylinder_fixation_yoke_fixation,
                'PAC': actuator_data.pre_adjustment_in_cylinder,
                'G_dimension': actuator_data.gdimension,
                'HTL': actuator_data.hanger_upper_bolt_to_lower_bolt,
                'BFH': actuator_data.hanger_lower_bolt_to_pad_holder_assemble_surface,
                'DHB': actuator_data.dis_between_hanger_upper_bolt,
                'HUTC': actuator_data.hanger_upper_bolt_to_caliper_center,
                'HLPC': actuator_data.hanger_lower_bolt_to_pad_center,
            }
            self.kwargs.update(additional_kwargs)

            return ClassicActCalculator(**self.kwargs)


        elif actuator_type == 'compact':
            additional_kwargs = {
                'LL': actuator_data.ll,
                'LR': actuator_data.lr,
                'LR1': actuator_data.lr1,
                'LR2': actuator_data.lr2,
                'D1': actuator_data.d1,
                'D2': actuator_data.d2,
                'D3': actuator_data.d3,
                'H1': actuator_data.h1,
                'H2': actuator_data.h2,
                'H3': actuator_data.h3,
                'Side': self.bogie_brake_system.extraSettings.sideReference,
            }
            self.kwargs.update(additional_kwargs)

            return CompactActCalculator(**self.kwargs)
        
    @property
    def pad_shape(self):
        return json.loads(self.pad_object.pad_shape)       
    
    @property
    def F_point(self):
        item_name=self.pad_object.item_name
        return float(item_name.split("_")[-1])

@dataclass
class OneStepSetttings(Component):
    oneStepInputs: dict

    def __post_init__(self):
        try:
            inputs = self.oneStepInputs["oneStep"]
            self.wheelSettingMode = inputs["wheelSettingMode"]
            self.verticalMove = inputs["verticalMove"]
            self.lateralMove = inputs["lateralMove"]
            self.longMove = inputs["longMove"]
            self.padWear = inputs["padWear"]
            self.discWear = inputs["discWear"]
            self.fPointOffset = inputs["fPointOffset"]
            self.brakingApplied = inputs["brakingApplied"]
            """ must ask frontend add this data field !!!"""
            self.padHolderBoltWear = 3.27
            self.leverBoltWear = 2

        except KeyError as e:
            print(f"Missing key in form data: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

class OneStepCalculation(BXCalculationModeBase):
    def __init__(self,bxCalculationSystem:BXCalculationSystem) -> None:
        super().__init__()
        self.bxCalculationSystem=bxCalculationSystem
        self.mode_inputs=OneStepSetttings(self.bxCalculationSystem.bogie_brake_system.functionalInputs.inputs)

    def calculation_results(self):

        """ 
            Basic Inputs for Static Positioning and Dynamic 
        """

        "static position"
        E_dimension = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.eDimension
        angle = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.rotationOfCaliper
        z_offset = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.installationOffsetZ
        y_offset = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.installationOffsetY
        clearance = self.bxCalculationSystem.bogie_brake_system.pad.clearance

        "dynamic movement"
        'movement'
        x_move= self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.X_LongitudinalMovement
        y_move = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.Y_LateralMovement
        'wear'
        disc_wear = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.discWear
        pad_wear = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.padWear

        "disc selection"
        disc_inner=self.bxCalculationSystem.bogie_brake_system.disc.discInnerDiameter
        disc_outer = self.bxCalculationSystem.bogie_brake_system.disc.discOuterDiameter
        Disc_Width = self.bxCalculationSystem.bogie_brake_system.disc.discWidth

        "pad selection"
        Pad_thickness = self.bxCalculationSystem.bogie_brake_system.pad.padThickness
        shape_data=self.bxCalculationSystem.pad_shape


        """ 
            Specific Settings and offsets calculation due to caliper movement/wear
        """

        "on setp setting with movement and wear"
        f_point_offset= self.mode_inputs.fPointOffset
        x_move_instant= self.mode_inputs.longMove
        y_move_instant= self.mode_inputs.lateralMove
        z_move_instant= self.mode_inputs.verticalMove
        pad_wear_instant= self.mode_inputs.padWear
        disc_wear_instant= self.mode_inputs.discWear
        PHBW_instant= self.mode_inputs.padHolderBoltWear
        LBW_instant= self.mode_inputs.leverBoltWear

        "offset due to caliper movement/wear"
        instantdata=self.bxCalculationSystem.actuator_calculator.transfer_list_instant( 
            y_move_instant=y_move_instant, disc_wear_instant=disc_wear_instant, pad_wear_instant=pad_wear_instant, WLB_instant=LBW_instant, WPHB_instant=PHBW_instant
            )
        D_w_l_Instant=instantdata.D_w_l_Instant
        Delta_Z_Instant=instantdata.Delta_Z_Instant
        Delta_Z = instantdata.Delta_Z

        """
        Calculation for this mode
        """

        "Inital position calculation"
        F_point=self.bxCalculationSystem.F_point
        E=E_dimension-f_point_offset
        F_offset=E-F_point
        fix_x=F_offset
        angle = angle * math.pi / 180

        points_group=[]
        n_area=int(shape_data[0][3])
        shape_data_list=[[] for i in range(n_area)]
        base_initial = [[] for i in range(n_area)]
        initial_group = [[] for i in range(n_area)]
        n_lower=4+n_area

        for i in range(0, n_area):
            n_points=int(shape_data[0][4+i])
            points_group.append(n_points)
            for j in range(0,n_points):
                shape_data_list[i].append([shape_data[0][j+n_lower]+fix_x, shape_data[1][j+n_lower]])
                shape_x0 = shape_data[0][j + n_lower] + fix_x
                shape_y0 = shape_data[1][j + n_lower]
                shape_x = shape_x0 * math.cos(angle) + (shape_y0 + Delta_Z) * math.sin(angle)
                shape_y = -1 * shape_x0 * math.sin(angle) + (shape_y0 + Delta_Z) * math.cos(angle)
                base_initial[i].append([shape_x, shape_y])
            n_lower=n_lower+n_points

        "Friction Radius of this step"
        E_delta_Rm = D_w_l_Instant - (z_offset + z_move_instant) * math.sin(angle) + x_move_instant * math.cos(angle)
        V_Rm = (z_offset + z_move_instant) * math.cos(angle) + x_move_instant * math.sin(angle) + Delta_Z_Instant
        R_m=EclipseCalculationUtils.Rm_cal(points_group, shape_data_list, E_dimension, E_delta_Rm, V_Rm)
        # R_m=Rm_cal(points_group, shape_data_list, E_dimension, E_delta_Rm, V_Rm)
            
        "Final Position of this step"
        n_area = len(points_group)
        for i in range(0,n_area):
            n_points = points_group[i]
            for j in range(0, n_points):
                shape_x = base_initial[i][j][0]+D_w_l_Instant*math.cos(angle)+Delta_Z_Instant* math.sin(angle)+x_move_instant
                shape_y = base_initial[i][j][1]-D_w_l_Instant * math.sin(angle)+Delta_Z_Instant *math.cos(angle)+z_offset+z_move_instant
                initial_group[i].append([shape_x,shape_y])

        "Extreme inner and outter points"
        initial_innermost =EclipseCalculationUtils.innermost(initial_group)
        initial_outermost =EclipseCalculationUtils.outermost(initial_group)

        "Circles of disc and friction radius of this step"
        circle_outer = EclipseCalculationUtils.circle_array(disc_outer)
        circle_inner = EclipseCalculationUtils.circle_array(disc_inner)
        circle_rm=EclipseCalculationUtils.circle_array(R_m*2)

        "Calculation Outputs"
        inner_outer_points={
                            "initial_innermost":initial_innermost,"initial_outermost": initial_outermost,
                            }
        data_group={
                    "very_original_shape_data":shape_data_list,
                    "n_area":n_area,
                    "points_group":points_group,
                    "initial_group":initial_group,
                    "circle_outer":circle_outer,
                    "circle_inner":circle_inner,
                    "R_m":R_m,
                    "circle_rm":circle_rm,
                    "inner_outer_points":inner_outer_points,
                    "E_delta_Rm": [E_delta_Rm,V_Rm,x_move_instant,math.cos(angle),z_offset,z_move_instant,math.sin(angle)]
                    }

        return data_group

@dataclass
class OdysseySetttings(Component):
    odysseyInputs: dict

    def __post_init__(self):
        try:
            inputs = self.odysseyInputs["odyssey"]
            """ must ask frontend add this data field !!!"""
            self.wheelInnerCheck = True
        except KeyError as e:
            print(f"Missing key in form data: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")

class OdysseyCalculation(BXCalculationModeBase):
    def __init__(self,bxCalculationSystem:BXCalculationSystem) -> None:
        super().__init__()
        self.bxCalculationSystem=bxCalculationSystem
        self.mode_inputs=OdysseySetttings(self.bxCalculationSystem.bogie_brake_system.functionalInputs.inputs)

    def calculation_results(self):

        """ 
            Basic Inputs for Static Positioning and Dynamic 
        """

        "static position"
        E_dimension = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.eDimension
        angle = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.rotationOfCaliper
        z_offset = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.installationOffsetZ
        y_offset = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.installationOffsetY
        clearance = self.bxCalculationSystem.bogie_brake_system.pad.clearance

        "dynamic movement"
        'movement'
        x_move= self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.X_LongitudinalMovement
        y_move = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.Y_LateralMovement
        lift_stop = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.A_TareDatumToLiftStop
        bump_stop = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.B_TareDatumToBumpStop
        crush = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.C_TareDatumToCrushDatum
        'wear'
        disc_wear = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.discWear
        pad_wear = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.padWear

        "disc selection"
        disc_inner=self.bxCalculationSystem.bogie_brake_system.disc.discInnerDiameter
        disc_outer = self.bxCalculationSystem.bogie_brake_system.disc.discOuterDiameter
        Disc_Width = self.bxCalculationSystem.bogie_brake_system.disc.discWidth
        wheel_screw = self.bxCalculationSystem.bogie_brake_system.wheel.wheelInnerDiameter
        "pad selection"
        Pad_thickness = self.bxCalculationSystem.bogie_brake_system.pad.padThickness
        shape_data=self.bxCalculationSystem.pad_shape

        """ 
            Specific Settings and offsets calculation due to caliper movement/wear
        """

        "offset due to caliper movement/wear"

        instantdata=self.bxCalculationSystem.actuator_calculator.transfer_list

        E_other = instantdata.E_other
        Delta_Z = instantdata.Delta_Z
        D_w_l_min_L0=instantdata.D_w_l_min_L0
        D_w_l_max_L0 = instantdata.D_w_l_max_L0
        D_w_l_min = instantdata.D_w_l_min
        D_w_l_max = instantdata.D_w_l_max
        Delta_Zmin=instantdata.Delta_Zmin
        Delta_Zmax = instantdata.Delta_Zmax

        """
        Calculation for this mode
        """

        "Inital position calculation"
        F_point=self.bxCalculationSystem.F_point
        E=E_dimension
        F_offset=E-F_point
        fix_x=F_offset
        angle = angle * math.pi / 180


        points_group=[]
        n_area=int(shape_data[0][3])
        shape_data_list=[[] for i in range(n_area)]
        base_initial = [[] for i in range(n_area)]
        initial_group = [[] for i in range(n_area)]
        tare_e_min_group = [[] for i in range(n_area)]
        tare_e_max_group = [[] for i in range(n_area)]
        crush_e_min_group = [[] for i in range(n_area)]
        crush_e_max_group = [[] for i in range(n_area)]
        wheel_inner_e_min_group = [[] for i in range(n_area)]
        wheel_inner_e_max_group = [[] for i in range(n_area)]
        lift_e_min_group = [[] for i in range(n_area)]
        lift_e_max_group = [[] for i in range(n_area)]
        bump_e_min_group = [[] for i in range(n_area)]
        bump_e_max_group = [[] for i in range(n_area)]
        n_lower=4+n_area
        
        for i in range(0, n_area):
            n_points=int(shape_data[0][4+i])
            points_group.append(n_points)
            for j in range(0,n_points):
                shape_data_list[i].append([shape_data[0][j+n_lower]+fix_x, shape_data[1][j+n_lower]])
                shape_x0 = shape_data[0][j + n_lower] + fix_x
                shape_y0 = shape_data[1][j + n_lower]
                shape_x = shape_x0 * math.cos(angle) + (shape_y0 + Delta_Z) * math.sin(angle)
                shape_y = -1 * shape_x0 * math.sin(angle) + (shape_y0 + Delta_Z) * math.cos(angle)
                base_initial[i].append([shape_x, shape_y])
            n_lower=n_lower+n_points

        "Friction Radius in initial state"
        R_m=EclipseCalculationUtils.Rm_cal(points_group, shape_data_list, E_dimension, 0, 0)
            
        "Generate different positions for multiple cases"
        n_area = len(points_group)
        for i in range(0,n_area):
            n_points = points_group[i]
            for j in range(0, n_points):

                shape_x=base_initial[i][j][0]
                shape_y=base_initial[i][j][1]+z_offset
                initial_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_min_L0*math.cos(angle)
                shape_y = base_initial[i][j][1] + z_offset- D_w_l_min_L0 * math.sin(angle)
                tare_e_min_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_max_L0*math.cos(angle)
                shape_y = base_initial[i][j][1] + z_offset- D_w_l_max_L0 * math.sin(angle)
                tare_e_max_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_min_L0*math.cos(angle)
                shape_y = base_initial[i][j][1] + z_offset- D_w_l_min_L0 * math.sin(angle)-crush
                crush_e_min_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_max_L0*math.cos(angle)
                shape_y = base_initial[i][j][1] + z_offset- D_w_l_max_L0 * math.sin(angle)-crush
                crush_e_max_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_min*math.cos(angle)+Delta_Zmin* math.sin(angle)-abs(x_move)
                shape_y = base_initial[i][j][1]-D_w_l_min * math.sin(angle)+Delta_Zmin *math.cos(angle)+z_offset+abs(lift_stop)
                lift_e_min_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_max*math.cos(angle)+Delta_Zmax* math.sin(angle)+abs(x_move)
                shape_y = base_initial[i][j][1]-D_w_l_max * math.sin(angle)+Delta_Zmax *math.cos(angle)+z_offset+abs(lift_stop)
                lift_e_max_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_min*math.cos(angle)+Delta_Zmin* math.sin(angle)-abs(x_move)
                shape_y = base_initial[i][j][1]-D_w_l_min * math.sin(angle)+Delta_Zmin *math.cos(angle)+z_offset-abs(bump_stop)
                bump_e_min_group[i].append([shape_x,shape_y])

                shape_x = base_initial[i][j][0]+D_w_l_max*math.cos(angle)+Delta_Zmax* math.sin(angle)+abs(x_move)
                shape_y = base_initial[i][j][1]-D_w_l_max * math.sin(angle)+Delta_Zmax *math.cos(angle)+z_offset-abs(bump_stop)
                bump_e_max_group[i].append([shape_x,shape_y])


        tare_innermost =EclipseCalculationUtils.innermost(tare_e_min_group)
        tare_outermost =EclipseCalculationUtils.outermost(tare_e_max_group)
        crush_innermost =EclipseCalculationUtils.innermost(crush_e_min_group)
        crush_outermost =EclipseCalculationUtils.outermost(crush_e_max_group)
        static_group = [tare_innermost, tare_outermost, crush_innermost, crush_outermost]
        static_innermost=min(static_group,key=lambda static_group:static_group[0]**2+static_group[1]**2)
        static_outermost = max(static_group, key=lambda static_group: static_group[0] ** 2 + static_group[1] ** 2)
        initial_innermost =EclipseCalculationUtils.innermost(initial_group)
        initial_outermost =EclipseCalculationUtils.outermost(initial_group)
        lift_innermost =EclipseCalculationUtils.innermost(lift_e_min_group)
        lift_outermost =EclipseCalculationUtils.outermost(lift_e_max_group)
        bump_innermost =EclipseCalculationUtils.innermost(bump_e_min_group)
        bump_outermost =EclipseCalculationUtils.outermost(bump_e_max_group)

        "Circles of disc and friction radius of Odyssey"
        circle_outer = EclipseCalculationUtils.circle_array(disc_outer)
        circle_inner = EclipseCalculationUtils.circle_array(disc_inner)
        circle_rm=EclipseCalculationUtils.circle_array(R_m*2)
        circle_wheel_screw=EclipseCalculationUtils.circle_array(wheel_screw)


        inner_outer_points={
                            "static_innermost":static_innermost,"static_outermost":static_outermost,
                            "initial_innermost":initial_innermost,"initial_outermost": initial_outermost,
                            "lift_innermost": lift_innermost, "lift_outermost": lift_outermost,
                            "bump_innermost": bump_innermost, "bump_outermost": bump_outermost
                            }

        data_group={
                    "very_original_shape_data":shape_data_list,
                    "n_area":n_area,
                    "points_group":points_group,
                    "initial_group":initial_group,
                    # "initial_group":base_initial_divided,
                    "circle_outer":circle_outer,
                    "circle_inner":circle_inner,
                    "circle_wheel_screw":circle_wheel_screw,
                    "wheel_screw":wheel_screw/2,
                    "R_m":R_m,
                    "circle_rm":circle_rm,
                    "E_other":E_other,
                    "tare_e_min_group":tare_e_min_group,
                    "tare_e_max_group":tare_e_max_group,
                    "crush_e_min_group":crush_e_min_group,
                    "crush_e_max_group":crush_e_max_group,
                    "lift_e_min_group": lift_e_min_group,
                    "lift_e_max_group": lift_e_max_group,
                    "bump_e_min_group": bump_e_min_group,
                    "bump_e_max_group": bump_e_max_group,
                    "inner_outer_points":inner_outer_points
                    }
        return data_group

class GraverCalculation(BXCalculationModeBase):
    def __init__(self,bxCalculationSystem:BXCalculationSystem) -> None:
        super().__init__()
        self.bxCalculationSystem=bxCalculationSystem

    def calculation_results(self):

        """
        Calculation Setting
        """
        P_tol=0.00001
        step_cal=1

        """ 
            Basic Inputs for Static Positioning and Dynamic 
        """

        "static position"
        E_dimension = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.eDimension
        angle = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.rotationOfCaliper
        z_offset = self.bxCalculationSystem.bogie_brake_system.brakeCaliperPositioning.installationOffsetZ
        "dynamic movement"
        'movement'
        x_move= self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.X_LongitudinalMovement
        crush = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.C_TareDatumToCrushDatum
        Tare_amp = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.E_AmplitudeTare
        Crush_amp = self.bxCalculationSystem.bogie_brake_system.kinematicPrimarySuspension.D_AmplitudeCrush

        "disc selection"
        disc_inner=self.bxCalculationSystem.bogie_brake_system.disc.discInnerDiameter
        disc_outer = self.bxCalculationSystem.bogie_brake_system.disc.discOuterDiameter
        Disc_Width = self.bxCalculationSystem.bogie_brake_system.disc.discWidth

        "pad selection"

        shape_data=self.bxCalculationSystem.pad_shape

        """ 
            Specific Settings and offsets calculation due to caliper movement/wear
        """

        "offset due to caliper movement/wear"

        instantdata=self.bxCalculationSystem.actuator_calculator.transfer_list
        Delta_Z = instantdata.Delta_Z
        D_w_l_min = instantdata.D_w_l_min
        D_w_l_max = instantdata.D_w_l_max

        """
        Calculation for this mode
        """

        "Inital position calculation"
        F_point=self.bxCalculationSystem.F_point
        F_offset=E_dimension-F_point
        angle = angle * math.pi / 180
        Long_1=x_move

        P_tol_divide = 0.08
        list = shape_data
        n_area = int(list[0][3])
        points_group = []
        points_group_divided = []
        base_initial_no_rotate = [[] for i in range(n_area)]

        n_lower = 4 + n_area
        base_initial_divided = [[] for i in range(n_area)]
        for i in range(0, n_area):
            n_points = int(list[0][4 + i])
            points_group.append(n_points)
            for j in range(0, n_points):
                base_initial_no_rotate[i].append([list[0][j + n_lower] + F_offset, list[1][j + n_lower]])
            n_lower = n_lower + n_points

        i = 0
        j = 1

        while i < n_area:
            n_points = points_group[i]
            PX1 = base_initial_no_rotate[i][0][0]
            PY1 = base_initial_no_rotate[i][0][1]
            shape_x = PX1 * math.cos(angle) + (PY1 + Delta_Z) * math.sin(angle)
            shape_y = -1 * PX1 * math.sin(angle) + (PY1 + Delta_Z) * math.cos(angle)
            base_initial_divided[i].append([shape_x, shape_y])
            points_group_divided.append(1)
            while j < n_points - 1:
                PX0 = base_initial_no_rotate[i][j - 1][0]
                PY0 = base_initial_no_rotate[i][j - 1][1]
                PX1 = base_initial_no_rotate[i][j][0]
                PY1 = base_initial_no_rotate[i][j][1]
                PX2 = base_initial_no_rotate[i][j + 1][0]
                PY2 = base_initial_no_rotate[i][j + 1][1]
                if abs((PX2 - PX1) * (PX1 - PX0)) > P_tol_divide ** 2:
                    if abs((PY2 - PY1) / (PX2 - PX1) - (PY1 - PY0) / (PX1 - PX0)) > P_tol_divide:
                        shape_x = PX1 * math.cos(angle) + (PY1 + Delta_Z) * math.sin(angle)
                        shape_y = -1 * PX1 * math.sin(angle) + (PY1 + Delta_Z) * math.cos(angle)
                        base_initial_divided[i].append([shape_x, shape_y])
                        points_group_divided[i] = points_group_divided[i] + 1
                        if (abs(PX2 - PX1) + abs(PY2 - PY1)) < 0.0001:
                            j = j + 1
                else:
                    if abs(PX2 - PX1) + abs(PX1 - PX0) > P_tol_divide:
                        shape_x = PX1 * math.cos(angle) + (PY1 + Delta_Z) * math.sin(angle)
                        shape_y = -1 * PX1 * math.sin(angle) + (PY1 + Delta_Z) * math.cos(angle)
                        base_initial_divided[i].append([shape_x, shape_y])
                        points_group_divided[i] = points_group_divided[i] + 1
                        if (abs(PX2 - PX1) + abs(PY2 - PY1)) < 0.0001:
                            j = j + 1
                j += 1
            PX1 = base_initial_no_rotate[i][n_points - 1][0]
            PY1 = base_initial_no_rotate[i][n_points - 1][1]
            shape_x = PX1 * math.cos(angle) + (PY1 + Delta_Z) * math.sin(angle)
            shape_y = -1 * PX1 * math.sin(angle) + (PY1 + Delta_Z) * math.cos(angle)
            base_initial_divided[i].append([shape_x, shape_y])
            points_group_divided[i] = points_group_divided[i] + 1
            j = 1
            i += 1

        R_step = []
        upper = int((disc_outer / 2 - round(disc_inner / 2)) * (1 / step_cal) + 1)
        for i in range(upper):
            R_step.append(disc_inner / 2 + i)
        r_x = [[0, 0] for i in range(len(R_step))]

        E_step = []
        upper = round((D_w_l_max - D_w_l_min) * (1 / step_cal) + 1)
        upper = int(upper)
        for i in range(upper):
            E_step.append(round(D_w_l_min + i * step_cal))

        NX_step = []
        Nx_min = -1 * abs(Long_1) + D_w_l_min * math.cos(angle)
        Nx_max = abs(Long_1) + D_w_l_max * math.cos(angle)
        upper = round((Nx_max - Nx_min) * (1 / step_cal) + 1)
        upper = int(upper)

        for i in range(upper):
            NX_step.append(round(Nx_min + i * step_cal))

        NTare_amp_step = []
        Nx_min = -1 * abs(Tare_amp) - D_w_l_max * math.sin(angle)
        Nx_max = abs(Tare_amp) - D_w_l_min * math.sin(angle)
        upper = round((Nx_max - Nx_min) * (1 / step_cal) + 1)
        upper = int(upper)
        for i in range(upper):
            NTare_amp_step.append(round(Nx_min + i * step_cal))

        NCrush_amp_step = []
        Nx_min = -1 * abs(Crush_amp) - D_w_l_max * math.sin(angle) - abs(crush)
        Nx_max = abs(Crush_amp) - D_w_l_min * math.sin(angle) - abs(crush)
        upper = round((Nx_max - Nx_min) * (1 / step_cal) + 1)
        upper = int(upper)
        for i in range(upper):
            NCrush_amp_step.append(round(Nx_min + i * step_cal))

        Group_step = [R_step, E_step, NX_step, NTare_amp_step, NCrush_amp_step]

        Group_X_Set = []
        Sum_X = 0
        Error_set = []

        for i_TTC in range(2):
            for i_x in range(len(NX_step)):
                # for i_x in range(1):
                #     print(NX_step[i_x])
                if i_TTC == 0:
                    Znn = len(NTare_amp_step)
                else:
                    Znn = len(NCrush_amp_step)
                for i_Z in range(Znn):
                    if i_TTC == 0:
                        Z_ap = NTare_amp_step[i_Z]
                    else:
                        Z_ap = NCrush_amp_step[i_Z]
                    P_x_d_t2 = [[] for i in range(n_area)]
                    for i_area in range(n_area):
                        n_points = points_group_divided[i_area]
                        for j_point in range(n_points):
                            shape_x = base_initial_divided[i_area][j_point][0] + NX_step[i_x]
                            shape_y = base_initial_divided[i_area][j_point][1] + Z_ap + z_offset
                            P_x_d_t2[i_area].append([shape_x, shape_y])
                    for I in range(len(R_step)):
                        r_x[I][0] = R_step[I]
                        State_P = 0
                        kkkkk = 0
                        Group_X_Set = []
                        while (State_P == 0):
                            for i_area in range(n_area):
                                n_points = points_group_divided[i_area]
                                for j_point in range(n_points - 1):
                                    Group_X = self.Intersection_Q(
                                                                    P_x_d_t2[i_area][j_point][0],
                                                                    P_x_d_t2[i_area][j_point][1],
                                                                    P_x_d_t2[i_area][j_point + 1][0],
                                                                    P_x_d_t2[i_area][j_point + 1][1],
                                                                    R_step[I], P_tol / 10)
                                    if Group_X[0] == 1000:
                                        R_step[I] = P_tol * 100 + R_step[I]
                                        # print(R_step[I])
                                        kkkkk = 0
                                        break
                                    if Group_X[0] != -1000:
                                        kkkkk = kkkkk + 1
                                        # if R_step[I] == 247.002 or R_step[I]==247 :
                                        #     if i_x == 2 and i_Z == 9:
                                        #         print(f"Group:{Group_X} R:{R_step[I]}")
                                        Group_X_Set.extend(Group_X)
                            if len(Group_X_Set) % 2 == 0:
                                State_P = 1
                            else:
                                Error_set.append(R_step[I])
                                R_step[I] = P_tol * 100 + R_step[I]
                                # print(R_step[I])
                                # print(Group_X_Set)
                                Group_X_Set = []

                        if kkkkk == 0:
                            Sum_X = 0
                        else:
                            Group_X_Set.sort(reverse=True)
                            Sum_X = 0

                            for count_x in range(0, len(Group_X_Set), 2):
                                Sum_X = Sum_X + Group_X_Set[count_x] - Group_X_Set[count_x + 1]

                            PPP = 0
                            if i_TTC == 0:
                                for E_j in range(len(E_step)):
                                    PPP = PPP + self.P_XYZ(0, Long_1,NX_step[i_x] - E_step[E_j] * math.cos(angle)) * \
                                          self.P_XYZ(0, Tare_amp,Z_ap + E_step[E_j] * math.sin(angle)) * \
                                          abs(1 / len(E_step)) * step_cal
                            else:
                                for E_j in range(len(E_step)):
                                    PPP = PPP + self.P_XYZ(0, Long_1,NX_step[i_x] - E_step[E_j] * math.cos(angle)) * \
                                          self.P_XYZ(0, Crush_amp,Z_ap + E_step[E_j] * math.sin(angle)) * \
                                          abs(1 / len(E_step)) * step_cal
                            r_x[I][1] = Sum_X * PPP * step_cal + r_x[I][1]

        V_T = 40000000
        V_wear = 0
        for I in range(len(r_x)):
            V_wear = V_wear + 2 * 3.14 * r_x[I][0] * r_x[I][1]

        V_T_T = V_T / V_wear
        V_wear = 0

        min_temp = 0
        for I in range(len(r_x)):
            r_x[I][1] = r_x[I][1] * V_T_T
            V_wear = V_wear + 2 * 3.14 * r_x[I][0] * r_x[I][1]
            r_x[I][1] = -r_x[I][1] / 3 - 45
            if r_x[I][1] < min_temp:
                min_temp = r_x[I][1]
            r_x[I][1] = r_x[I][1] + 300

        sum_D = round(abs(min_temp + 45) * 3 / 39, 2)

        list_result = {
            "n_area": n_area,
            "points_group": points_group,
            "base_initial_divided": base_initial_divided,
            "points_group_divided": points_group_divided,
            "R_step": R_step,
            "r_x": r_x,
            "Group_step": Group_step,
            "sum_D": sum_D,
            "Error_set": Error_set,
            "Error_set_len": len(Error_set)
        }

        return list_result

    @staticmethod
    def Intersection_Q(PPX1,PPY1,PPX2,PPY2,RRk,E_A):
        T_Temp=[]
        PPA = (PPX2 - PPX1)** 2 + (PPY2 - PPY1) ** 2
        PPB = 2 * PPX1 * (PPX2 - PPX1) + 2 * PPY1 * (PPY2 - PPY1)
        PPC = PPX1 ** 2 + PPY1 ** 2 - RRk ** 2
        Delta_I = PPB ** 2 - 4 * PPA * PPC
        if Delta_I<0:
            T_Temp.append(-1000)
        elif Delta_I<=E_A and Delta_I >= 0:
            PPt1 = -PPB / (2 * PPA)
            if PPt1>1 or PPt1 < 0:
                T_Temp.append(-1000)
            elif PPt1 <= 1 and PPt1 >= 1 - E_A or PPt1 >= 0 and PPt1 <= E_A:
                T_Temp.append(1000)
            else:
                Deg_1=math.atan((PPY1 + PPt1 * (PPY2 - PPY1)) / (PPX1 + PPt1 * (PPX2 - PPX1)))
                T_Temp.append(Deg_1*180/math.pi)
                T_Temp.append(Deg_1 * 180 / math.pi)
        else:
            PPt1 = 1 / (2 * PPA) * (-PPB - Delta_I ** 0.5)
            PPt2 = 1 / (2 * PPA) * (-PPB + Delta_I ** 0.5)
            if PPt1 <= 1 and PPt1 >= 1 - E_A or PPt1 >= 0 and PPt1 <= E_A or PPt2 <= 1 and PPt2 >= 1 - E_A or PPt2 >= 0 and PPt2 <= E_A:
                T_Temp.append(1000)
            elif (PPt1 > 1 or PPt1 < 0) and (PPt2 > 1 or PPt2 < 0):
                T_Temp.append(-1000)
            elif (PPt1 < 1 - E_A and PPt1 > E_A) and (PPt2 > 1 or PPt2 < 0):
                Deg_1 = math.atan((PPY1 + PPt1 * (PPY2 - PPY1)) / (PPX1 + PPt1 * (PPX2 - PPX1)))
                T_Temp.append(Deg_1 * 180 / math.pi)
            elif (PPt2 < 1 - E_A and PPt2 > E_A) and (PPt1 > 1 or PPt1 < 0):
                Deg_1 = math.atan((PPY1 + PPt2 * (PPY2 - PPY1)) / (PPX1 + PPt2 * (PPX2 - PPX1)))
                T_Temp.append(Deg_1 * 180 / math.pi)
            elif (PPt2 < 1 - E_A and PPt2 > E_A) and (PPt1 < 1 - E_A and PPt1 > E_A):
                Deg_1 = math.atan((PPY1 + PPt1 * (PPY2 - PPY1)) / (PPX1 + PPt1 * (PPX2 - PPX1)))
                Deg_2 = math.atan((PPY1 + PPt2 * (PPY2 - PPY1)) / (PPX1 + PPt2 * (PPX2 - PPX1)))
                T_Temp.append(Deg_1 * 180 / math.pi)
                T_Temp.append(Deg_2 * 180 / math.pi)
        return T_Temp
    @staticmethod
    def P_XYZ(PU, AM, xyz):
        if xyz<-1*AM+PU or xyz>AM+PU:
            normal_num=0
        else:
            e = 2.7182818
            pi = 3.1415926
            standard_dev=AM/1.6
            normal_num = e ** (-((xyz - PU) ** 2) / (2 * standard_dev ** 2)) / (standard_dev * (2 * pi) ** 0.5)
        return normal_num




In [ ]:
onstepCalculationSystem=BXCalculationSystem(json_data)
OneStepCalculation(onstepCalculationSystem).calculation_results()

In [ ]:
odysseyCalculationSystem=BXCalculationSystem(json_data)
OdysseyCalculation(odysseyCalculationSystem).calculation_results()

In [ ]:
graverCalculationSystem=BXCalculationSystem(json_data)
GraverCalculation(graverCalculationSystem).calculation_results()